<a href="https://colab.research.google.com/github/kwarkmc/tensorflow_training/blob/main/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import platform

In [ ]:
platform.platform()

In [ ]:
!pip install -q seaborn

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

In [ ]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

In [ ]:
len(dataset)

In [ ]:
dataset.head()

In [ ]:
dataset = dataset.dropna()

In [ ]:
len(dataset)

In [ ]:
dataset.pop('Origin')

In [ ]:
dataset

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
len(train_dataset), len(test_dataset)

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [ ]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
normed_train_data.head()

In [ ]:
normed_train_data.shape

In [ ]:
inputs = keras.Input(shape=(normed_train_data.shape[1],))
h = layers.Dense(64) (inputs)
h = layers.Activation('relu')(h)
h = layers.Dense(32) (h)
outputs = layers.Dense(1) (h)

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.RMSprop(0.001))

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "my_first_model_with_shape_info.png", show_shapes=True)

In [ ]:
normed_train_data[0:5].shape

In [ ]:
example_batch = normed_train_data[:5]
example_result = model.predict(example_batch)
example_result

In [ ]:
EPOCHS = 100
history = model.fit(normed_train_data, train_labels, epochs=EPOCHS, validation_split=0.2)

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(normed_train_data, train_labels, epochs=EPOCHS, validation_split= 0.2, callbacks=[early_stop])

In [ ]:
loss = model.evaluate(normed_test_data, test_labels, verbose=2)

print("테스트 세트의 평균 절대 오차 : {:5.2f} MPG".format(loss))

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Valies [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_=plt.plot([-100,100], [-100,100])